In [1]:
from pandas import DataFrame
import pandas as pd
import numpy as np

## Preprocess

In [2]:
# Load the datas
product = pd.read_csv('Data_01.Pruduct.NoADDNoHit.csv', dtype={"CLNT_ID": np.str, "SESS_ID": np.str, "PD_C": np.str}, thousands=",", usecols = [0,1,2,3,5,6])
custom = pd.read_csv('Data_04.Custom.csv',dtype={"CLNT_ID": np.str})
session = pd.read_csv('Data_05.Session.Day.csv',thousands=",", dtype={"CLNT_ID": np.str, "SESS_ID": np.str})
master = pd.read_csv('Data_06.Master.Col.csv', dtype={"PD_C": np.str},usecols=[0,2,3,4,5])

In [3]:
product.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_BUY_AM,PD_BUY_CT
0,4139680,07605037,12,642112,39000,1
1,4140076,10189797,13,570603,39000,2
2,4142395,06158159,85,179538,39000,1
3,4144914,07935714,12,554336,39000,1
4,4144917,06406509,78,190306,39000,5


In [4]:
m1 = pd.merge(product, custom, on='CLNT_ID', how = 'inner')

In [5]:
del product

In [6]:
session.dropna(inplace=True)
session['SESS_DT'] = pd.to_datetime(session['SESS_DT'], format="%Y%m%d")
session['SESS_month'] = pd.DatetimeIndex(session['SESS_DT']).month
session.head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,weekday,weekday/weekend,SESS_month
0,5873599,08641867,9,2018-05-09,82.0,1890.0,mobile,Daejeon,Daejeon,Wednesday,weekday,5
1,5873599,06616320,21,2018-06-11,105.0,1604.0,mobile,Busan,Busan,Monday,weekday,6
2,5873599,05886172,40,2018-06-24,41.0,632.0,mobile,Daejeon,Daejeon,Sunday,weekend,6
3,5873884,01050889,15,2018-09-13,160.0,1035.0,mobile,Gyeonggi-do,Anyang,Thursday,weekday,9
4,5874461,10298270,5,2018-04-12,13.0,298.0,mobile,Seoul,Seoul,Thursday,weekday,4


In [7]:
#categorical variable to dummy
categorical_features = ['DVC_CTG_NM', 'ZON_NM', 'SESS_month','weekday','weekday/weekend']
prefix_name = ["DVC","ZON","month",'day','weekend']

for i in range(len(prefix_name)):
    dummies = pd.get_dummies(session[categorical_features[i]], prefix=prefix_name[i])
    session = pd.concat([session, dummies], axis=1)
    session.drop(categorical_features[i], axis=1, inplace=True)
session.head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,CITY_NM,DVC_desktop,DVC_mobile,DVC_tablet,...,month_9,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,weekend_weekday,weekend_weekend
0,5873599,08641867,9,2018-05-09,82.0,1890.0,Daejeon,0,1,0,...,0,0,0,0,0,0,0,1,1,0
1,5873599,06616320,21,2018-06-11,105.0,1604.0,Busan,0,1,0,...,0,0,1,0,0,0,0,0,1,0
2,5873599,05886172,40,2018-06-24,41.0,632.0,Daejeon,0,1,0,...,0,0,0,0,1,0,0,0,0,1
3,5873884,01050889,15,2018-09-13,160.0,1035.0,Anyang,0,1,0,...,1,0,0,0,0,1,0,0,1,0
4,5874461,10298270,5,2018-04-12,13.0,298.0,Seoul,0,1,0,...,0,0,0,0,0,1,0,0,1,0


In [8]:
session.drop(['CITY_NM'], axis=1, inplace=True)

In [9]:
m1 = pd.merge(m1, session, on=['CLNT_ID','SESS_ID'], how = 'inner')

In [10]:
labels, uniques = pd.factorize(master['CLAC0_NM'],sort=True)
print(uniques)

Index(['가구/침구/인테리어', '가전/디지털', '반려동물', '스포츠/레저', '식품', '여행/서비스/금융', '유아동',
       '주방/생활/건강', '패션의류', '화장품/뷰티'],
      dtype='object')


In [11]:
dummies = pd.get_dummies(labels, prefix='CLAC0')
master = pd.concat([master, dummies], axis=1)
master.drop(["CLAC0_NM","CLAC1_NM","CLAC2_NM","CLAC3_NM"], axis=1, inplace=True)
master.head()

,PD_C,CLAC0_0,CLAC0_1,CLAC0_2,CLAC0_3,CLAC0_4,CLAC0_5,CLAC0_6,CLAC0_7,CLAC0_8,CLAC0_9
0,064382,0,0,0,1,0,0,0,0,0,0
1,062282,0,0,0,0,0,0,0,0,1,0
2,061729,0,0,0,0,0,0,1,0,0,0
3,061537,0,0,0,0,0,0,0,0,1,0
4,058820,0,0,0,0,0,0,0,0,1,0


In [12]:
m1 = pd.merge(m1, master, on=['PD_C'], how = 'inner')
m1.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_BUY_AM,PD_BUY_CT,CLNT_GENDER,CLNT_AGE,SESS_SEQ,SESS_DT,...,CLAC0_0,CLAC0_1,CLAC0_2,CLAC0_3,CLAC0_4,CLAC0_5,CLAC0_6,CLAC0_7,CLAC0_8,CLAC0_9
0,4140076,10189797,13,570603,39000,2,F,40,172,2018-04-13,...,0,0,0,1,0,0,0,0,0,0
1,4148106,06537318,21,570603,39000,1,F,30,58,2018-06-13,...,0,0,0,1,0,0,0,0,0,0
2,4148106,06662803,94,570603,39000,1,F,30,55,2018-06-11,...,0,0,0,1,0,0,0,0,0,0
3,5298612,04443505,129,570603,39000,1,F,30,710,2018-07-17,...,0,0,0,1,0,0,0,0,0,0
4,5553084,06453463,55,570603,39000,1,F,30,772,2018-06-14,...,0,0,0,1,0,0,0,0,0,0


In [13]:
m1['CLAC0_0'] = m1['CLAC0_0']*m1['PD_BUY_CT']
m1['CLAC0_1'] = m1['CLAC0_1']*m1['PD_BUY_CT']
m1['CLAC0_2'] = m1['CLAC0_2']*m1['PD_BUY_CT']
m1['CLAC0_3'] = m1['CLAC0_3']*m1['PD_BUY_CT']
m1['CLAC0_4'] = m1['CLAC0_4']*m1['PD_BUY_CT']
m1['CLAC0_5'] = m1['CLAC0_5']*m1['PD_BUY_CT']
m1['CLAC0_6'] = m1['CLAC0_6']*m1['PD_BUY_CT']
m1['CLAC0_7'] = m1['CLAC0_7']*m1['PD_BUY_CT']
m1['CLAC0_8'] = m1['CLAC0_8']*m1['PD_BUY_CT']
m1['CLAC0_9'] = m1['CLAC0_9']*m1['PD_BUY_CT']
m1['TOTAL_BUY_AM'] = m1['PD_BUY_AM']*m1['PD_BUY_CT']

In [14]:
m1.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_BUY_AM,PD_BUY_CT,CLNT_GENDER,CLNT_AGE,SESS_SEQ,SESS_DT,...,CLAC0_1,CLAC0_2,CLAC0_3,CLAC0_4,CLAC0_5,CLAC0_6,CLAC0_7,CLAC0_8,CLAC0_9,TOTAL_BUY_AM
0,4140076,10189797,13,570603,39000,2,F,40,172,2018-04-13,...,0,0,2,0,0,0,0,0,0,78000
1,4148106,06537318,21,570603,39000,1,F,30,58,2018-06-13,...,0,0,1,0,0,0,0,0,0,39000
2,4148106,06662803,94,570603,39000,1,F,30,55,2018-06-11,...,0,0,1,0,0,0,0,0,0,39000
3,5298612,04443505,129,570603,39000,1,F,30,710,2018-07-17,...,0,0,1,0,0,0,0,0,0,39000
4,5553084,06453463,55,570603,39000,1,F,30,772,2018-06-14,...,0,0,1,0,0,0,0,0,0,39000


In [15]:
m1.columns

Index(['CLNT_ID', 'SESS_ID', 'HITS_SEQ', 'PD_C', 'PD_BUY_AM', 'PD_BUY_CT',
       'CLNT_GENDER', 'CLNT_AGE', 'SESS_SEQ', 'SESS_DT', 'TOT_PAG_VIEW_CT',
       'TOT_SESS_HR_V', 'DVC_desktop', 'DVC_mobile', 'DVC_tablet', 'ZON_Busan',
       'ZON_Chungcheongbuk-do', 'ZON_Chungcheongnam-do', 'ZON_Daegu',
       'ZON_Daejeon', 'ZON_Gangwon-do', 'ZON_Gwangju', 'ZON_Gyeonggi-do',
       'ZON_Gyeongsangbuk-do', 'ZON_Gyeongsangnam-do', 'ZON_Incheon',
       'ZON_Jeju-do', 'ZON_Jeollabuk-do', 'ZON_Jeollanam-do', 'ZON_Seoul',
       'ZON_Ulsan', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8',
       'month_9', 'day_Friday', 'day_Monday', 'day_Saturday', 'day_Sunday',
       'day_Thursday', 'day_Tuesday', 'day_Wednesday', 'weekend_weekday',
       'weekend_weekend', 'CLAC0_0', 'CLAC0_1', 'CLAC0_2', 'CLAC0_3',
       'CLAC0_4', 'CLAC0_5', 'CLAC0_6', 'CLAC0_7', 'CLAC0_8', 'CLAC0_9',
       'TOTAL_BUY_AM'],
      dtype='object')

In [16]:
# Client 최초 등장일 & 활동 기간
base_date = pd.to_datetime('2018-10-01')
first_date = m1[["CLNT_ID", "SESS_DT"]].groupby(["CLNT_ID"]).min() #최초 등장 일자
first_date = first_date.reset_index()
first_date = first_date.rename(columns={"SESS_DT": "FIRST_PURCHASE"})
first_date["ACTIVE_DAYS"] = first_date["FIRST_PURCHASE"].apply(lambda x: pd.Timedelta(base_date - x).days) #기준 일자 - 최초 등장 일자로 활동기간 계산
first_date.head()

,CLNT_ID,FIRST_PURCHASE,ACTIVE_DAYS
0,0000008,2018-06-05,118
1,0000014,2018-09-11,20
2,0000032,2018-09-11,20
3,0000056,2018-06-26,97
4,0000058,2018-07-19,74


In [17]:
# 고객 당 세션 수, 구매 제품 종류, 접속일 수
m1_unique = m1.groupby(['CLNT_ID']).nunique()[['SESS_ID',"PD_C",'SESS_DT']]
m1_unique = m1_unique.reset_index()
m1_unique = m1_unique.rename(columns={"SESS_ID":"N_SESS","PD_C": "N_PRODUCT","SESS_DT": "N_LOGIN"})
m1_unique.head()

,CLNT_ID,N_SESS,N_PRODUCT,N_LOGIN
0,0000008,2,1,2
1,0000014,1,2,1
2,0000032,2,4,2
3,0000056,2,2,2
4,0000058,1,5,1


In [18]:
m2 = pd.merge(first_date,m1_unique,on=['CLNT_ID'])
del first_date,m1_unique
# Client 별 평균 활동 주기 (활동 기간 / 접속일 수)
m2["PURCHASE_CYCLE"] = m2["ACTIVE_DAYS"] / m2["N_LOGIN"]
m2.head()

,CLNT_ID,FIRST_PURCHASE,ACTIVE_DAYS,N_SESS,N_PRODUCT,N_LOGIN,PURCHASE_CYCLE
0,0000008,2018-06-05,118,2,1,2,59.0
1,0000014,2018-09-11,20,1,2,1,20.0
2,0000032,2018-09-11,20,2,4,2,10.0
3,0000056,2018-06-26,97,2,2,2,48.5
4,0000058,2018-07-19,74,1,5,1,74.0


In [19]:
# Client 별 페이지뷰, 체류시간, 제품가격, 소비액 평균
m1_mean = m1.groupby(['CLNT_ID'])['HITS_SEQ','SESS_SEQ','TOT_PAG_VIEW_CT','TOT_SESS_HR_V','PD_BUY_AM','TOTAL_BUY_AM'].agg(['mean'])
m1_mean.columns = m1_mean.columns.get_level_values(0)
m1_mean.sort_values('TOTAL_BUY_AM',ascending=False).head()

,HITS_SEQ,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,PD_BUY_AM,TOTAL_BUY_AM
CLNT_ID,,,,,,
0929827,57.000000,11.000000,51.0,1974.000000,1.627350e+07,1.627350e+07
1103732,32.500000,17.500000,48.5,1159.500000,5.780000e+06,8.670000e+06
5780703,53.000000,4.000000,82.0,2980.000000,1.405000e+06,7.727500e+06
6132785,137.333333,33.777778,189.0,3301.222222,4.430546e+06,7.346834e+06
6320318,65.500000,8.000000,105.5,1088.000000,4.732500e+06,6.217500e+06


In [20]:
# 총 소비액, 구매 갯수 
m1_sum = m1.groupby(['CLNT_ID'])['TOTAL_BUY_AM','PD_BUY_CT'].agg(['sum'])
m1_sum.columns = m1_sum.columns.get_level_values(0)
m1_sum = m1_sum.rename(columns={"PD_BUY_CT":"TOTAL_CT"})
m1_sum.head()

,TOTAL_BUY_AM,TOTAL_CT
CLNT_ID,,
0000008,76000,2
0000014,69800,2
0000032,76800,5
0000056,23800,3
0000058,131100,6


In [21]:
# Client 별 사용 기기 종류 비율
DVC = m1.loc[:, m1.columns.str.startswith('DVC')]
DVC_sum = pd.concat([m1['CLNT_ID'], DVC], axis=1).groupby(['CLNT_ID']).agg(['sum'])
DVC_sum.columns = DVC_sum.columns.get_level_values(0)
DVC_sum = DVC_sum.div(DVC_sum.sum(axis=1),axis=0)
DVC_sum.head()

,DVC_desktop,DVC_mobile,DVC_tablet
CLNT_ID,,,
0000008,0.0,1.0,0.0
0000014,0.0,1.0,0.0
0000032,0.0,1.0,0.0
0000056,0.0,1.0,0.0
0000058,0.0,1.0,0.0


In [22]:
# Client 별 구매 지역 비율
ZON = m1.loc[:, m1.columns.str.startswith('ZON')]
ZON_sum = pd.concat([m1['CLNT_ID'], ZON], axis=1).groupby(['CLNT_ID']).agg(['sum'])
ZON_sum.columns = ZON_sum.columns.get_level_values(0)
ZON_sum = ZON_sum.div(ZON_sum.sum(axis=1),axis=0)
ZON_sum.head()

,ZON_Busan,ZON_Chungcheongbuk-do,ZON_Chungcheongnam-do,ZON_Daegu,ZON_Daejeon,ZON_Gangwon-do,ZON_Gwangju,ZON_Gyeonggi-do,ZON_Gyeongsangbuk-do,ZON_Gyeongsangnam-do,ZON_Incheon,ZON_Jeju-do,ZON_Jeollabuk-do,ZON_Jeollanam-do,ZON_Seoul,ZON_Ulsan
CLNT_ID,,,,,,,,,,,,,,,,
0000008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0
0000014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0000032,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0000056,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0000058,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# Client 별 구매 달 비율
month = m1.loc[:, m1.columns.str.startswith('month')]
month_sum = pd.concat([m1['CLNT_ID'], month], axis=1).groupby(['CLNT_ID']).agg(['sum'])
month_sum.columns = month_sum.columns.get_level_values(0)
month_sum = month_sum.div(month_sum.sum(axis=1),axis=0)
month_sum.head()

,month_4,month_5,month_6,month_7,month_8,month_9
CLNT_ID,,,,,,
0000008,0.0,0.0,1.000000,0.0,0.000000,0.0
0000014,0.0,0.0,0.000000,0.0,0.000000,1.0
0000032,0.0,0.0,0.000000,0.0,0.000000,1.0
0000056,0.0,0.0,0.333333,0.0,0.666667,0.0
0000058,0.0,0.0,0.000000,1.0,0.000000,0.0


In [24]:
# Client 별 구매 날짜 비율
day = m1.loc[:, m1.columns.str.startswith('day')]
day_sum = pd.concat([m1['CLNT_ID'], day], axis=1).groupby(['CLNT_ID']).agg(['sum'])
day_sum.columns = day_sum.columns.get_level_values(0)
day_sum = day_sum.div(day_sum.sum(axis=1),axis=0)
day_sum.head()

,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday
CLNT_ID,,,,,,,
0000008,0.0,0.0,0.0,0.000000,0.0,0.500000,0.5
0000014,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0
0000032,0.0,0.4,0.0,0.000000,0.0,0.600000,0.0
0000056,0.0,0.0,0.0,0.666667,0.0,0.333333,0.0
0000058,0.0,0.0,0.0,0.000000,1.0,0.000000,0.0


In [25]:
# Client 별 구매 주말 비율
weekend = m1.loc[:, m1.columns.str.startswith('weekend')]
weekend_sum = pd.concat([m1['CLNT_ID'], weekend], axis=1).groupby(['CLNT_ID']).agg(['sum'])
weekend_sum.columns = weekend_sum.columns.get_level_values(0)
weekend_sum = weekend_sum.div(weekend_sum.sum(axis=1),axis=0)
weekend_sum.head()

,weekend_weekday,weekend_weekend
CLNT_ID,,
0000008,0.500000,0.500000
0000014,1.000000,0.000000
0000032,1.000000,0.000000
0000056,0.333333,0.666667
0000058,1.000000,0.000000


In [26]:
# Client 별 category 구매 갯수
CLAC0 = m1.loc[:, m1.columns.str.startswith('CLAC0') ]
CLAC0_sum = pd.concat([m1['CLNT_ID'], CLAC0], axis=1).groupby(['CLNT_ID']).agg(['sum'])
CLAC0_sum.columns = CLAC0_sum.columns.get_level_values(0)
CLAC0_sum.head()

,CLAC0_0,CLAC0_1,CLAC0_2,CLAC0_3,CLAC0_4,CLAC0_5,CLAC0_6,CLAC0_7,CLAC0_8,CLAC0_9
CLNT_ID,,,,,,,,,,
0000008,0,0,0,0,0,0,0,0,2,0
0000014,0,0,0,0,0,0,0,0,2,0
0000032,0,0,0,0,0,0,0,0,5,0
0000056,0,0,0,0,0,0,1,0,0,2
0000058,0,0,0,1,0,0,0,0,5,0


In [27]:
m2 = m2.drop(['FIRST_PURCHASE'], axis=1)
m2 = pd.merge(m2,m1_mean,on=['CLNT_ID'])
m2 = pd.merge(m2,m1_sum,on=['CLNT_ID'])
m2 = pd.merge(m2,DVC_sum,on='CLNT_ID')
m2 = pd.merge(m2,ZON_sum,on=['CLNT_ID'])
m2 = pd.merge(m2,month_sum,on=['CLNT_ID'])
m2 = pd.merge(m2,day_sum,on=['CLNT_ID'])
m2 = pd.merge(m2,weekend_sum,on=['CLNT_ID'])
m2 = pd.merge(m2,CLAC0_sum,on=['CLNT_ID'])

In [32]:
m2 = pd.merge(m2,custom,on=['CLNT_ID'])
m2.head()

,CLNT_ID,ACTIVE_DAYS,N_SESS,N_PRODUCT,N_LOGIN,PURCHASE_CYCLE,HITS_SEQ,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,...,CLAC0_2,CLAC0_3,CLAC0_4,CLAC0_5,CLAC0_6,CLAC0_7,CLAC0_8,CLAC0_9,CLNT_GENDER,CLNT_AGE
0,0000008,118,2,1,2,59.0,57.000000,4.5,46.500000,417.000000,...,0,0,0,0,0,0,2,0,M,20
1,0000014,20,1,2,1,20.0,284.000000,5.0,256.000000,7098.000000,...,0,0,0,0,0,0,2,0,F,30
2,0000032,20,2,4,2,10.0,58.000000,101.4,98.200000,2169.000000,...,0,0,0,0,0,0,5,0,M,40
3,0000056,97,2,2,2,48.5,47.333333,82.0,45.666667,1157.666667,...,0,0,0,0,1,0,0,2,F,40
4,0000058,74,1,5,1,74.0,209.000000,8.0,195.000000,3161.000000,...,0,1,0,0,0,0,5,0,F,30


In [31]:
custom.head()

,CLNT_ID,CLNT_GENDER,CLNT_AGE
0,4830726,F,30
1,4830874,F,40
2,4830975,F,30
3,4831275,F,30
4,4825325,F,30


In [33]:
m2.to_csv('data_regression.csv',index=False)